In [1]:
# DAFTAR lIBRARY

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import optuna
import os
import logging
import time
import warnings
import matplotlib.dates as mdates
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
from matplotlib.dates import DateFormatter
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
from uuid import uuid4
from torch.utils.data import random_split




In [2]:
df = pd.read_csv('student_monnitoring_data.csv')

In [3]:
display((df))
display(df.info())
display(df.describe())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level
0,1,2024-12-01,9:00-15:00,Late,0.92,7.6,6,6,Low
1,1,2024-12-02,8:00-16:00,Late,1.17,6.0,6,2,Medium
2,1,2024-12-03,11:00-14:00,Late,4.56,6.3,4,8,High
3,1,2024-12-04,11:00-16:00,Late,3.07,9.0,2,10,Low
4,1,2024-12-05,9:00-13:00,Absent,3.93,7.4,9,4,High
...,...,...,...,...,...,...,...,...,...
14995,500,2024-12-26,9:00-16:00,Late,1.30,7.2,7,10,Low
14996,500,2024-12-27,9:00-15:00,Absent,1.07,7.9,4,6,High
14997,500,2024-12-28,11:00-14:00,Absent,1.67,7.2,3,5,High
14998,500,2024-12-29,11:00-14:00,Late,0.99,7.2,10,9,Medium


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Student ID          15000 non-null  int64  
 1   Date                15000 non-null  object 
 2   Class Time          15000 non-null  object 
 3   Attendance Status   15000 non-null  object 
 4   Stress Level (GSR)  15000 non-null  float64
 5   Sleep Hours         15000 non-null  float64
 6   Anxiety Level       15000 non-null  int64  
 7   Mood Score          15000 non-null  int64  
 8   Risk Level          15000 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 1.0+ MB


None

,Student ID,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score
count,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000
mean,250.50000,2.762538,6.996780,5.546867,5.471533
std,144.34209,1.301927,1.150973,2.870323,2.868984
min,1.00000,0.500000,5.000000,1.000000,1.000000
25%,125.75000,1.640000,6.000000,3.000000,3.000000
50%,250.50000,2.760000,7.000000,6.000000,5.000000
75%,375.25000,3.900000,8.000000,8.000000,8.000000
max,500.00000,5.000000,9.000000,10.000000,10.000000


In [4]:
# Pengecekan Missing value pada setiap kolom
missing_values = df.isna().sum()
print("Jumlah Missing Value pada setiap kolom:")
print(missing_values)

# Pengecekan Presentasi Missing value per kolom
missing_percentage = (df.isna().sum() / len(df)) * 100
print("\nPersentase Missing value pada setiap Kolim:")
print(missing_percentage)


Jumlah Missing Value pada setiap kolom:
Student ID            0
Date                  0
Class Time            0
Attendance Status     0
Stress Level (GSR)    0
Sleep Hours           0
Anxiety Level         0
Mood Score            0
Risk Level            0
dtype: int64

Persentase Missing value pada setiap Kolim:
Student ID            0.0
Date                  0.0
Class Time            0.0
Attendance Status     0.0
Stress Level (GSR)    0.0
Sleep Hours           0.0
Anxiety Level         0.0
Mood Score            0.0
Risk Level            0.0
dtype: float64


In [5]:
# Menghitung Z-Score untuk kolom-kolom numerik
import numpy as np
z_scores = zscore(df[['Stress Level (GSR)', 'Anxiety Level', 'Mood Score', 'Sleep Hours']])

# Menyaring data dengan Z-Score > 3 atau < -3
outliers_z = np.where(abs(z_scores) > 3)  # Menemukan posisi outlier berdasarkan Z-Score

# Menampilkan data outlier berdasarkan Z-Score
outliers_z_data = df.iloc[outliers_z[0]]
print("Outliers detected by Z-Score:")
print(outliers_z_data)


Outliers detected by Z-Score:
Empty DataFrame
Columns: [Student ID, Date, Class Time, Attendance Status, Stress Level (GSR), Sleep Hours, Anxiety Level, Mood Score, Risk Level]
Index: []


In [6]:
# Menghitung Z-Score per user
def calculate_zscore_per_user(df, columns):
    outliers = []  # Untuk menyimpan hasil outlier per user
    for user_id in df['Student ID'].unique():  # Loop melalui setiap user
        user_data = df[df['Student ID'] == user_id][columns]
        z_scores = zscore(user_data, axis=0)  # Menghitung Z-Score per kolom untuk user tertentu

        # Mencari outlier untuk Z-Score > 3 atau < -3
        outliers_for_user = user_data[(abs(z_scores) > 3).any(axis=1)]  # Filter baris dengan Z-Score > 3
        outliers.append(outliers_for_user)

    # Gabungkan semua outliers per user
    return pd.concat(outliers)

# Tentukan kolom yang ingin dianalisis (kolom numerik)
columns_to_check = ['Stress Level (GSR)', 'Anxiety Level', 'Mood Score', 'Sleep Hours']

# Temukan outliers untuk seluruh dataset
outliers_z_data_per_user = calculate_zscore_per_user(df, columns_to_check)

print("Outliers detected by Z-Score per user:")
print(outliers_z_data_per_user)


Outliers detected by Z-Score per user:
Empty DataFrame
Columns: [Stress Level (GSR), Anxiety Level, Mood Score, Sleep Hours]
Index: []


In [7]:
# Mengonversi kolom 'Date' menjadi format datetime
df['Date'] = pd.to_datetime(df['Date'])
print(df['Date'].head())
df['Day'] = df['Date'].dt.day

df = df.sort_values(by=['Student ID', 'Date'])

user_day_count = df.groupby('Student ID')['Date'].count()

0   2024-12-01
1   2024-12-02
2   2024-12-03
3   2024-12-04
4   2024-12-05
Name: Date, dtype: datetime64[ns]


In [8]:
# label Encoding untuk Risk Level
le = LabelEncoder()
df['Risk Level'] = le.fit_transform(df['Risk Level'])


# label Encoder Untuk ateendande Status
attendance_mapping = {'Present': 0, 'Absent': 1, 'Late': 2}
df['Attendance Status (Data)'] = df['Attendance Status'].map(attendance_mapping)
df['Attendance Status (Data)'] = df['Attendance Status (Data)'].astype('Int64')
df[['Attendance Status', 'Attendance Status (Data)']].head()
display(df.head())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data)
0,1,2024-12-01,9:00-15:00,Late,0.92,7.6,6,6,1,1,2
1,1,2024-12-02,8:00-16:00,Late,1.17,6.0,6,2,2,2,2
2,1,2024-12-03,11:00-14:00,Late,4.56,6.3,4,8,0,3,2
3,1,2024-12-04,11:00-16:00,Late,3.07,9.0,2,10,1,4,2
4,1,2024-12-05,9:00-13:00,Absent,3.93,7.4,9,4,0,5,1


In [9]:

# Memilih kolom untuk Min-Max Scaling dan Standardization
cols_min_max = ['Stress Level (GSR)', 'Anxiety Level', 'Mood Score']
cols_standardize = ['Sleep Hours']

# Membuat objek scaler untuk Min-Max dan Standardization
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Melakukan Min-Max Scaling pada kolom yang dipilih
df[cols_min_max] = min_max_scaler.fit_transform(df[cols_min_max])

# Melakukan Standardization pada kolom yang dipilih
df[cols_standardize] = standard_scaler.fit_transform(df[cols_standardize])

# Menampilkan hasil setelah scaling dan standardisasi
display(df.head())


,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data)
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1


In [10]:
# Lag Features untuk Stress Level (GSR) dan Sleep Hours
df['lag_1_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].shift(1)
df['lag_1_sleep'] = df.groupby('Student ID')['Sleep Hours'].shift(1)

# Rolling Statistics untuk Stress Level (GSR) dengan window size 3 (misalnya 3 hari)
df['rolling_mean_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].rolling(window=3).mean().reset_index(0, drop=True)
df['rolling_std_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].rolling(window=3).std().reset_index(0, drop=True)

# Menampilkan hasil setelah menambahkan lag features dan rolling statistics
display(df.head())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data),lag_1_stress,lag_1_sleep,rolling_mean_stress,rolling_std_stress
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2,NaN,NaN,NaN,NaN
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2,0.093333,0.524113,NaN,NaN
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2,0.148889,-0.866061,0.381481,0.451829
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2,0.902222,-0.605403,0.540741,0.377584
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1,0.571111,1.740515,0.745185,0.166212


In [11]:
from statsmodels.tsa.stattools import adfuller

# Fungsi untuk uji ADF pada setiap Student ID
def check_stationarity(data, col):
    result = adfuller(df[col].dropna())
    return result[1]  # Mengembalikan p-value

# Menguji stasioneritas untuk setiap Student ID
p_values = df.groupby('Student ID').apply(lambda group: check_stationarity(group, 'Stress Level (GSR)'))


C:\Users\MSI\AppData\Local\Temp\ipykernel_7948\23598078.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  p_values = df.groupby('Student ID').apply(lambda group: check_stationarity(group, 'Stress Level (GSR)'))


In [12]:


# Fungsi untuk mengecek stasioneritas
def check_stationarity(data, col):
    result = adfuller(data[col].dropna())
    return result[1]  # Mengembalikan p-value

# Fungsi untuk melakukan differencing
def difference_data(data, col, d=1):
    data_copy = data.copy()
    for _ in range(d):
        data_copy[col] = data_copy[col].diff()
    return data_copy

# Inisialisasi variabel
stationary_ids = []
non_stationary_ids = []
p_values_initial = {}
p_values_after_diff = {}
df_processed = []

# Loop per Student ID
for student_id in df['Student ID'].unique():
    data_sub = df[df['Student ID'] == student_id].copy()

    # Cek stasioneritas awal
    p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
    p_values_initial[student_id] = p_value

    if p_value <= 0.05:
        # Sudah stasioner
        stationary_ids.append(student_id)
        p_values_after_diff[student_id] = p_value
    else:
        # Belum stasioner, lakukan differencing pertama
        data_sub = difference_data(data_sub, 'Stress Level (GSR)', d=1)
        p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
        p_values_after_diff[student_id] = p_value

        if p_value <= 0.05:
            stationary_ids.append(student_id)
        else:
            # Lakukan differencing kedua
            data_sub = difference_data(data_sub, 'Stress Level (GSR)', d=2)
            p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
            p_values_after_diff[student_id] = p_value

            if p_value <= 0.05:
                stationary_ids.append(student_id)
            else:
                non_stationary_ids.append(student_id)

    df_processed.append(data_sub)

# Tampilkan daftar hasil akhir
print("\n===== HASIL UJI STASIONERITAS =====")
print("Student ID yang STASIONER:")
print(stationary_ids)

print("\nStudent ID yang BELUM STASIONER:")
print(non_stationary_ids)



===== HASIL UJI STASIONERITAS =====
Student ID yang STASIONER:
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 2

In [13]:
# Menggunakan interpolasi linear untuk mengisi NaN setelah differencing
df[['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score',
      'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']] = \
    df[['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score',
          'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']].interpolate(method='linear')

# Menampilkan data setelah interpolasi
df.head()


,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data),lag_1_stress,lag_1_sleep,rolling_mean_stress,rolling_std_stress
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2,NaN,NaN,NaN,NaN
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2,0.093333,0.524113,NaN,NaN
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2,0.148889,-0.866061,0.381481,0.451829
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2,0.902222,-0.605403,0.540741,0.377584
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1,0.571111,1.740515,0.745185,0.166212


In [14]:
# Menghitung jumlah observasi asli untuk setiap Student ID
jumlah_observasi = df.groupby('Student ID').size()

print("\n===== JUMLAH OBSERVASI PER STUDENT ID =====")
for student_id, jumlah in jumlah_observasi.items():
    print(f"Student ID {student_id}: {jumlah} observasi")



===== JUMLAH OBSERVASI PER STUDENT ID =====
Student ID 1: 30 observasi
Student ID 2: 30 observasi
Student ID 3: 30 observasi
Student ID 4: 30 observasi
Student ID 5: 30 observasi
Student ID 6: 30 observasi
Student ID 7: 30 observasi
Student ID 8: 30 observasi
Student ID 9: 30 observasi
Student ID 10: 30 observasi
Student ID 11: 30 observasi
Student ID 12: 30 observasi
Student ID 13: 30 observasi
Student ID 14: 30 observasi
Student ID 15: 30 observasi
Student ID 16: 30 observasi
Student ID 17: 30 observasi
Student ID 18: 30 observasi
Student ID 19: 30 observasi
Student ID 20: 30 observasi
Student ID 21: 30 observasi
Student ID 22: 30 observasi
Student ID 23: 30 observasi
Student ID 24: 30 observasi
Student ID 25: 30 observasi
Student ID 26: 30 observasi
Student ID 27: 30 observasi
Student ID 28: 30 observasi
Student ID 29: 30 observasi
Student ID 30: 30 observasi
Student ID 31: 30 observasi
Student ID 32: 30 observasi
Student ID 33: 30 observasi
Student ID 34: 30 observasi
Student ID 3

In [15]:
#Menyimpan dataset baru setelah preprocessing 
df.to_csv('dataset_baru.csv', index=False)


In [20]:


warnings.filterwarnings("ignore")

# =============================================================================
# BAGIAN 1: FUNGSI DAN KELAS MODEL (DIMODIFIKASI DENGAN MULTI-HEAD ATTENTION)
# =============================================================================

def setup_logging(model_dir, experiment_name):
    """Mengatur logging untuk menyimpan output ke file."""
    os.makedirs(model_dir, exist_ok=True)
    log_file = os.path.join(model_dir, f'log_{experiment_name}.log')
    # Hapus handler lama jika ada untuk menghindari duplikasi log
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

# MODIFIKASI: Fungsi MAPE dihapus sesuai permintaan
# def mean_absolute_percentage_error(y_true, y_pred):
#     """Menghitung Mean Absolute Percentage Error (MAPE)."""
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     non_zero_mask = y_true != 0
#     if not np.any(non_zero_mask): return 0.0
#     return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

def create_sequences(data, student_ids, target_cols, timesteps=3):
    """Membuat sekuens data untuk model time series."""
    X, X_ids, y = [], [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i + timesteps])
        X_ids.append(student_ids[i + timesteps])
        y.append(data[i + timesteps, -len(target_cols):])
    return np.array(X), np.array(X_ids), np.array(y)

# MODIFIKASI: Kelas Attention diganti dengan MultiHeadAttention wrapper
class MultiHeadAttention(nn.Module):
    """
    Wrapper untuk nn.MultiheadAttention dari PyTorch.
    Memungkinkan model untuk fokus pada timestep yang relevan dengan lebih efektif.
    """
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Pastikan hidden_dim dapat dibagi oleh num_heads
        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads"
        self.mha = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)

    def forward(self, lstm_outputs):
        """
        Args:
            lstm_outputs: Output dari layer LSTM, shape: (batch, seq_len, hidden_dim)
        Returns:
            context_vector: Vektor konteks hasil pembobotan, shape: (batch, hidden_dim)
            attn_weights: Bobot atensi, shape: (batch, seq_len)
        """
        # Query, Key, dan Value adalah output LSTM itu sendiri
        attn_output, attn_weights = self.mha(lstm_outputs, lstm_outputs, lstm_outputs)
        # Gunakan output dari attention layer sebagai context vector
        # Kita ambil output terakhir yang telah diberi bobot atensi, atau bisa juga di-average
        context_vector = attn_output[:, -1, :]
        return context_vector, attn_weights.mean(dim=1) # Rata-ratakan bobot dari semua head

class LSTMAttentionModel(nn.Module):
    """
    Model LSTM yang diintegrasikan dengan Multi-Head Attention Mechanism.
    """
    # MODIFIKASI: Menambahkan num_heads
    def __init__(self, input_dim, num_students, embed_dim, hidden_dim, num_heads, timesteps=3, dropout_rate=0.4):
        super(LSTMAttentionModel, self).__init__()
        self.embedding = nn.Embedding(num_students, embed_dim)
        self.lstm = nn.LSTM(input_dim + embed_dim, hidden_dim, batch_first=True)
        # MODIFIKASI: Menggunakan kelas MultiHeadAttention yang baru
        self.attention = MultiHeadAttention(hidden_dim, num_heads)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.risk_head = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(), nn.Dropout(dropout_rate - 0.1 if dropout_rate > 0.1 else 0.0), nn.Linear(hidden_dim // 2, 3))
        self.fc_residual = nn.Linear(hidden_dim, 1)

    def forward(self, x, student_ids):
        if student_ids.ndim == 1: student_ids = student_ids.unsqueeze(1)
        embed = self.embedding(student_ids.squeeze(1))
        embed = embed.unsqueeze(1).repeat(1, x.shape[1], 1)
        x = torch.cat([x, embed], dim=2)
        
        lstm_out, _ = self.lstm(x)
        
        # MODIFIKASI: Attention layer memproses output LSTM
        context_vector, _ = self.attention(lstm_out)
        
        out = self.bn(context_vector)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        residual_pred = self.fc_residual(out)
        return risk_logits, residual_pred.squeeze(1)

class LSTMModel(nn.Module):
    """
    Model LSTM standar tanpa Attention Mechanism.
    """
    def __init__(self, input_dim, num_students, embed_dim, hidden_dim, timesteps=3, dropout_rate=0.4):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(num_students, embed_dim)
        self.lstm = nn.LSTM(input_dim + embed_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.risk_head = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(), nn.Dropout(dropout_rate - 0.1 if dropout_rate > 0.1 else 0.0), nn.Linear(hidden_dim // 2, 3))
        self.fc_residual = nn.Linear(hidden_dim, 1)

    def forward(self, x, student_ids):
        if student_ids.ndim == 1: student_ids = student_ids.unsqueeze(1)
        embed = self.embedding(student_ids.squeeze(1))
        embed = embed.unsqueeze(1).repeat(1, x.shape[1], 1)
        x = torch.cat([x, embed], dim=2)

        lstm_out, (h_n, _) = self.lstm(x)
        last_hidden_state = h_n.squeeze(0)
        
        out = self.bn(last_hidden_state)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        residual_pred = self.fc_residual(out)
        return risk_logits, residual_pred.squeeze(1)


# def tune_arima(train_data, val_data, p_range=range(0, 5), d_range=range(0, 3), q_range=range(0, 5)):
#     """Mencari parameter ARIMA terbaik menggunakan time series cross-validation."""
#     best_mae = float('inf')
#     best_params = (1, 1, 1)
#     tscv = TimeSeriesSplit(n_splits=3)
#     for p in p_range:
#         for d in d_range:
#             for q in q_range:
#                 try:
#                     mae_list = []
#                     for train_index, val_index in tscv.split(train_data):
#                         train_fold, val_fold = train_data.iloc[train_index], train_data.iloc[val_index]
#                         model = ARIMA(train_fold['Stress Level (GSR)'], order=(p, d, q)).fit()
#                         predictions = model.forecast(steps=len(val_fold))
#                         mae = mean_absolute_error(val_fold['Stress Level (GSR)'], predictions)
#                         mae_list.append(mae)
#                     mean_mae = np.mean(mae_list)
#                     if mean_mae < best_mae:
#                         best_mae, best_params = mean_mae, (p, d, q)
#                 except Exception:
#                     continue
#     return best_params, best_mae

def tune_arima(train_data, val_data, p_range=range(0, 5), d_range=range(0, 3), q_range=range(0, 5)):
    best_mae = float('inf')
    best_params = (1, 1, 1)
    tscv = TimeSeriesSplit(n_splits=3)
    
    print("\n Proses tuning ARIMA dimulai...")
    
    for p in p_range:
        for d in d_range:
            for q in q_range:
                try:
                    mae_list = []
                    for train_index, val_index in tscv.split(train_data):
                        train_fold, val_fold = train_data.iloc[train_index], train_data.iloc[val_index]
                        model = ARIMA(train_fold['Stress Level (GSR)'], order=(p, d, q)).fit()
                        predictions = model.forecast(steps=len(val_fold))
                        mae = mean_absolute_error(val_fold['Stress Level (GSR)'], predictions)
                        mae_list.append(mae)
                    mean_mae = np.mean(mae_list)
                    print(f" - ARIMA({p},{d},{q}) → MAE: {mean_mae:.4f}")
                    if mean_mae < best_mae:
                        best_mae, best_params = mean_mae, (p, d, q)
                except Exception as e:
                    print(f"Gagal ARIMA({p},{d},{q}): {e}")
                    continue

    print("\n Parameter ARIMA terbaik:")
    print(f" - ARIMA{best_params} dengan MAE: {best_mae:.4f}")
    
    return best_params, best_mae


# def tune_lstm(X_train_tensor, X_train_ids_tensor, y_train_risk, y_train_residual, timesteps, num_students, device, use_attention):
#     """Mencari hyperparameter terbaik untuk model LSTM (dengan/tanpa Attention) menggunakan Optuna."""
#     def objective(trial):
#         # MODIFIKASI: Pastikan hidden_dim kelipatan dari num_heads (misal 8)
#         hidden_dim = trial.suggest_int('hidden_dim', 64, 256, step=8)
#         dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
#         lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
#         risk_weight = trial.suggest_float('risk_weight', 0.8, 2.0)
#         embed_dim = trial.suggest_int('embed_dim', 10, 50)
        
#         if use_attention:
#             # MODIFIKASI: Tambahkan num_heads ke tuning
#             num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
#             model = LSTMAttentionModel(input_dim=X_train_tensor.shape[2], num_students=num_students, embed_dim=embed_dim, hidden_dim=hidden_dim, num_heads=num_heads, timesteps=timesteps, dropout_rate=dropout_rate).to(device)
#         else:
#             model = LSTMModel(input_dim=X_train_tensor.shape[2], num_students=num_students, embed_dim=embed_dim, hidden_dim=hidden_dim, timesteps=timesteps, dropout_rate=dropout_rate).to(device)

#         loss_risk, loss_residual = nn.CrossEntropyLoss(), nn.MSELoss()
#         optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        
#         val_size = int(len(X_train_tensor) * 0.2)
#         X_train_lstm, X_val_lstm = X_train_tensor[:-val_size], X_train_tensor[-val_size:]
#         X_train_ids_lstm, X_val_ids_lstm = X_train_ids_tensor[:-val_size], X_train_ids_tensor[-val_size:]
#         y_train_risk_lstm, y_val_risk_lstm = y_train_risk[:-val_size], y_train_risk[-val_size:]
#         y_train_residual_lstm, y_val_residual_lstm = y_train_residual[:-val_size], y_train_residual[-val_size:]
        
#         train_dataset = TensorDataset(X_train_lstm, X_train_ids_lstm, y_train_risk_lstm, y_train_residual_lstm)
#         train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#         best_val_loss = float('inf')
        
#         for epoch in range(40):
#             model.train()
#             for batch_x, batch_ids, batch_y_risk, batch_y_residual in train_loader:
#                 batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
#                 optimizer.zero_grad()
#                 risk_logits, residual_pred = model(batch_x, batch_ids)
#                 loss = risk_weight * loss_risk(risk_logits, batch_y_risk) + 1.0 * loss_residual(residual_pred, batch_y_residual)
#                 loss.backward()
#                 optimizer.step()
            
#             model.eval()
#             with torch.no_grad():
#                 val_risk_logits, val_residual_pred = model(X_val_lstm, X_val_ids_lstm)
#                 val_loss = (risk_weight * loss_risk(val_risk_logits, y_val_risk_lstm) + 1.0 * loss_residual(val_residual_pred, y_val_residual_lstm)).item()
#                 if val_loss < best_val_loss: best_val_loss = val_loss
#         return best_val_loss
        
#     study = optuna.create_study(direction='minimize')
#     study.optimize(objective, n_trials=10, n_jobs=-1)
#     return study.best_params

def tune_lstm(X_train_tensor, X_train_ids_tensor, y_train_risk, y_train_residual, timesteps, num_students, device, use_attention):
    """Mencari hyperparameter terbaik untuk model LSTM (dengan/tanpa Attention) menggunakan Optuna."""
    def objective(trial):
        hidden_dim = trial.suggest_int('hidden_dim', 64, 256, step=8)
        dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
        lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
        risk_weight = trial.suggest_float('risk_weight', 0.8, 2.0)
        embed_dim = trial.suggest_int('embed_dim', 10, 50)
        
        if use_attention:
            num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
            model = LSTMAttentionModel(
                input_dim=X_train_tensor.shape[2],
                num_students=num_students,
                embed_dim=embed_dim,
                hidden_dim=hidden_dim,
                num_heads=num_heads,
                timesteps=timesteps,
                dropout_rate=dropout_rate
            ).to(device)
        else:
            model = LSTMModel(
                input_dim=X_train_tensor.shape[2],
                num_students=num_students,
                embed_dim=embed_dim,
                hidden_dim=hidden_dim,
                timesteps=timesteps,
                dropout_rate=dropout_rate
            ).to(device)

        loss_risk, loss_residual = nn.CrossEntropyLoss(), nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        
        val_size = int(len(X_train_tensor) * 0.2)
        X_train_lstm, X_val_lstm = X_train_tensor[:-val_size], X_train_tensor[-val_size:]
        X_train_ids_lstm, X_val_ids_lstm = X_train_ids_tensor[:-val_size], X_train_ids_tensor[-val_size:]
        y_train_risk_lstm, y_val_risk_lstm = y_train_risk[:-val_size], y_train_risk[-val_size:]
        y_train_residual_lstm, y_val_residual_lstm = y_train_residual[:-val_size], y_train_residual[-val_size:]
        
        train_dataset = TensorDataset(X_train_lstm, X_train_ids_lstm, y_train_risk_lstm, y_train_residual_lstm)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        best_val_loss = float('inf')
        
        for epoch in range(40):
            model.train()
            for batch_x, batch_ids, batch_y_risk, batch_y_residual in train_loader:
                batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
                optimizer.zero_grad()
                risk_logits, residual_pred = model(batch_x, batch_ids)
                loss = risk_weight * loss_risk(risk_logits, batch_y_risk) + 1.0 * loss_residual(residual_pred, batch_y_residual)
                loss.backward()
                optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_risk_logits, val_residual_pred = model(X_val_lstm, X_val_ids_lstm)
                val_loss = (risk_weight * loss_risk(val_risk_logits, y_val_risk_lstm) + 1.0 * loss_residual(val_residual_pred, y_val_residual_lstm)).item()
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
        return best_val_loss

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10, n_jobs=-1)

    best_params = study.best_params

    # 🔽 CETAK PARAMETER HASIL TUNING
    print("\n📌 Hyperparameter LSTM terbaik dari tuning:")
    for k, v in best_params.items():
        print(f" - {k}: {v}")
    
    return best_params


# =============================================================================
# BAGIAN 2: FUNGSI PERAMALAN DINAMIS (TIDAK BERUBAH)
# =============================================================================

def create_feature_forecasters(data, primary_feature_cols):
    """Membuat model peramalan sederhana untuk setiap fitur primer."""
    forecasters = {}
    for col in primary_feature_cols:
        try:
            model = ARIMA(data[col], order=(2,1,1)).fit()
            forecasters[col] = model
            logging.info(f"Feature forecaster created for {col}")
        except Exception as e:
            logging.error(f"Could not create forecaster for {col}: {e}")
            forecasters[col] = None
    return forecasters

def forecast_future(data, arima_stress_fit, model_lstm, le_student, feature_forecasters, timesteps, n_forecast, feature_cols, primary_feature_cols):
    """Meramalkan tingkat stres secara iteratif dengan meramalkan fitur inputnya juga."""
    logging.info("Memulai peramalan bertingkat yang dinamis...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_lstm.to(device).eval()

    arima_stress_forecast = arima_stress_fit.forecast(steps=n_forecast)
    all_forecasts = []

    for student_id in data['Student ID'].unique():
        student_history = data[data['Student ID'] == student_id].copy().reset_index(drop=True)
        if len(student_history) < timesteps:
            logging.warning(f"Skipping forecast for student {student_id} due to insufficient historical data ({len(student_history)} < {timesteps}).")
            continue
            
        student_id_encoded = le_student.transform([student_id])[0]
        
        student_future_predictions = []

        future_features_forecasts = {}
        for col, forecaster in feature_forecasters.items():
            if forecaster:
                future_features_forecasts[col] = forecaster.forecast(steps=n_forecast)
            else:
                future_features_forecasts[col] = [student_history[col].iloc[-1]] * n_forecast

        with torch.no_grad():
            for i in range(n_forecast):
                input_data = student_history.tail(timesteps)
                
                current_features = [col for col in feature_cols if col in input_data.columns]
                missing_features = set(feature_cols) - set(current_features)
                if missing_features:
                    logging.error(f"Missing features for forecast: {missing_features}")
                    break
                    
                input_features_sequence = input_data[current_features].values.astype(np.float32)
                
                if 'residual' in input_data.columns:
                    input_residuals_sequence = input_data['residual'].values.astype(np.float32).reshape(-1, 1)
                else:
                    input_residuals_sequence = np.zeros((timesteps, 1), dtype=np.float32)
                    
                input_aug = np.concatenate((input_features_sequence, input_residuals_sequence), axis=1)
                
                current_sequence = torch.tensor(input_aug, dtype=torch.float32).unsqueeze(0).to(device)
                current_id_tensor = torch.tensor([student_id_encoded], dtype=torch.long).to(device)

                _, lstm_residual_pred = model_lstm(current_sequence, current_id_tensor)
                lstm_residual_pred = lstm_residual_pred.cpu().item()
                
                final_stress_pred = arima_stress_forecast.iloc[i] + lstm_residual_pred
                student_future_predictions.append(final_stress_pred)

                new_row_data = {
                    'Date': student_history['Date'].iloc[-1] + pd.DateOffset(days=1),
                    'Student ID': student_id,
                    'Student ID Encoded': student_id_encoded,
                    'Stress Level (GSR)': final_stress_pred,
                    'residual': lstm_residual_pred,
                }
                for col in primary_feature_cols:
                    new_row_data[col] = future_features_forecasts[col].iloc[i] if hasattr(future_features_forecasts[col], 'iloc') else future_features_forecasts[col][i]

                new_row_df = pd.DataFrame([new_row_data])
                student_history = pd.concat([student_history, new_row_df], ignore_index=True)
                
                student_history['lag_1_stress'] = student_history['Stress Level (GSR)'].shift(1)
                if 'Sleep Hours' in student_history.columns:
                    student_history['lag_1_sleep'] = student_history['Sleep Hours'].shift(1)
                student_history['rolling_mean_stress'] = student_history['Stress Level (GSR)'].rolling(window=7, min_periods=1).mean()
                student_history['rolling_std_stress'] = student_history['Stress Level (GSR)'].rolling(window=7, min_periods=1).std()
                student_history.ffill(inplace=True)
                student_history.bfill(inplace=True)

        forecast_dates = pd.to_datetime([data[data['Student ID']==student_id]['Date'].max() + pd.DateOffset(days=j) for j in range(1, n_forecast + 1)])
        forecast_df_student = pd.DataFrame({'Date': forecast_dates, 'Student ID': student_id, 'Forecasted Stress Level': student_future_predictions})
        all_forecasts.append(forecast_df_student)

    if not all_forecasts: return None
    return pd.concat(all_forecasts, ignore_index=True)


# =============================================================================
# BAGIAN 3: FUNGSI UTAMA EKSPERIMEN
# =============================================================================

def run_experiment(data, experiment_name, model_dir, feature_cols, primary_feature_cols, use_attention, timesteps=3):
    """Menjalankan satu siklus eksperimen penuh: training, validasi, prediksi, dan peramalan."""
    setup_logging(model_dir, experiment_name)
    logging.info(f"===== MEMULAI EKSPERIMEN: {experiment_name} =====")
    logging.info(f"Menggunakan fitur: {feature_cols}")
    logging.info(f"Menggunakan Attention: {use_attention}")

    # ===== Persiapan Data =====
    all_required_cols = ['Student ID', 'Date', 'Stress Level (GSR)', 'Risk Level'] + feature_cols
    all_required_cols = sorted(list(set(all_required_cols)))

    if any(col not in data.columns for col in all_required_cols): 
        missing = [col for col in all_required_cols if col not in data.columns]
        raise ValueError(f"Kolom yang dibutuhkan hilang untuk eksperimen ini: {missing}")
    
    data_exp = data.copy()
    data_exp['Date'] = pd.to_datetime(data_exp['Date'])
    data_exp = data_exp.sort_values(by=['Student ID', 'Date']).reset_index(drop=True)
    
    le_student = LabelEncoder()
    data_exp['Student ID Encoded'] = le_student.fit_transform(data_exp['Student ID'])
    with open(os.path.join(model_dir, f'label_encoder_{experiment_name}.pkl'), 'wb') as f: pickle.dump(le_student, f)
    
    unique_dates = data_exp['Date'].dt.date.unique()
    if len(unique_dates) < 30: raise ValueError(f"Tanggal unik tidak cukup: {len(unique_dates)}")
    
    train_dates, test_dates = unique_dates[:24], unique_dates[24:30]
    train_data = data_exp[data_exp['Date'].dt.date.isin(train_dates)].copy().reset_index(drop=True)
    test_data = data_exp[data_exp['Date'].dt.date.isin(test_dates)].copy().reset_index(drop=True)
    if len(test_data) == 0 or len(train_data) == 0: raise ValueError("Data latih atau uji kosong")

    target_cols = ['Risk Level', 'Stress Level (GSR)']
    for col in feature_cols + target_cols:
        for df in [train_data, test_data]:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)
            df[col] = df.groupby('Student ID')[col].transform(lambda x: x.ffill().bfill()).fillna(df[col].mean())


    # ===== Pelatihan Model ARIMA =====
    logging.info("Membuat model peramal untuk fitur-fitur primer...")
    feature_forecasters = create_feature_forecasters(train_data, primary_feature_cols)
    with open(os.path.join(model_dir, f'feature_forecasters_{experiment_name}.pkl'), 'wb') as f: pickle.dump(feature_forecasters, f)

    arima_train_size = int(len(train_data) * 0.8)
    train_arima, val_arima = train_data.iloc[:arima_train_size], train_data.iloc[arima_train_size:]
    best_params_arima, best_mae_arima = tune_arima(train_arima, val_arima)
    logging.info(f"Parameter ARIMA global: {best_params_arima} dengan MAE: {best_mae_arima}")
    
    arima_stress_fit = ARIMA(train_data['Stress Level (GSR)'], order=best_params_arima).fit()
    with open(os.path.join(model_dir, f'arima_stress_model_{experiment_name}.pkl'), 'wb') as f: pickle.dump(arima_stress_fit, f)

    arima_train_pred = arima_stress_fit.predict(start=0, end=len(train_data) - 1)
    arima_test_pred = arima_stress_fit.forecast(steps=len(test_data))
    
    train_data['residual'] = train_data['Stress Level (GSR)'].values - arima_train_pred.values
    test_data['residual'] = test_data['Stress Level (GSR)'].values - arima_test_pred.values
    
    # ===== Persiapan Data untuk LSTM =====
    train_values = train_data[feature_cols + target_cols].values
    test_values = test_data[feature_cols + target_cols].values
    train_student_ids = train_data['Student ID Encoded'].values
    test_student_ids = test_data['Student ID Encoded'].values
    X_train, X_train_ids, y_train = create_sequences(train_values, train_student_ids, target_cols, timesteps)
    X_test, X_test_ids, y_test = create_sequences(test_values, test_student_ids, target_cols, timesteps)
    
    residual_train_seq = train_data.loc[timesteps:, 'residual'].values[:len(X_train)].astype(np.float32)
    residual_test_seq = test_data.loc[timesteps:, 'residual'].values[:len(X_test)].astype(np.float32)
    
    X_train_aug = np.concatenate((X_train[:, :, :len(feature_cols)], residual_train_seq[:, np.newaxis, np.newaxis].repeat(timesteps, axis=1)), axis=2).astype(np.float32)
    X_test_aug = np.concatenate((X_test[:, :, :len(feature_cols)], residual_test_seq[:, np.newaxis, np.newaxis].repeat(timesteps, axis=1)), axis=2).astype(np.float32)
    
    # ===== Pelatihan Model LSTM =====
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"Tuning LSTM (use_attention={use_attention})...")
    best_params_lstm = tune_lstm(torch.tensor(X_train_aug).to(device), torch.tensor(X_train_ids).to(device), torch.tensor(y_train[:, 0], dtype=torch.long).to(device), torch.tensor(residual_train_seq).to(device), timesteps, len(le_student.classes_), device, use_attention)
    logging.info(f"Parameter LSTM Terbaik: {best_params_lstm}")

    if use_attention:
        model_lstm = LSTMAttentionModel(input_dim=X_train_aug.shape[2], num_students=len(le_student.classes_), embed_dim=best_params_lstm['embed_dim'], hidden_dim=best_params_lstm['hidden_dim'], num_heads=best_params_lstm['num_heads'], timesteps=timesteps, dropout_rate=best_params_lstm['dropout_rate']).to(device)
    else:
        model_lstm = LSTMModel(input_dim=X_train_aug.shape[2], num_students=len(le_student.classes_), embed_dim=best_params_lstm['embed_dim'], hidden_dim=best_params_lstm['hidden_dim'], timesteps=timesteps, dropout_rate=best_params_lstm['dropout_rate']).to(device)

    optimizer = torch.optim.Adam(model_lstm.parameters(), lr=best_params_lstm['lr'], weight_decay=1e-5)
    
    # MODIFIKASI: Split data latih menjadi train & validation untuk plotting loss curve
    full_train_dataset = TensorDataset(torch.tensor(X_train_aug), torch.tensor(X_train_ids), torch.tensor(y_train[:, 0], dtype=torch.long), torch.tensor(residual_train_seq))
    val_size = int(len(full_train_dataset) * 0.2)
    train_size = len(full_train_dataset) - val_size
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    # MODIFIKASI: Variabel untuk menyimpan history loss
    train_losses, val_losses = [], []
    
    logging.info("Memulai pelatihan akhir model LSTM...")
    for epoch in range(50):
        model_lstm.train()
        epoch_train_loss = 0.0
        for batch_x, batch_ids, batch_y_risk, batch_y_residual in train_loader:
            batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
            optimizer.zero_grad()
            risk_logits, residual_pred = model_lstm(batch_x, batch_ids)
            loss = best_params_lstm['risk_weight'] * nn.CrossEntropyLoss()(risk_logits, batch_y_risk) + nn.MSELoss()(residual_pred, batch_y_residual)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        
        train_losses.append(epoch_train_loss / len(train_loader))

        # MODIFIKASI: Kalkulasi validation loss per epoch
        model_lstm.eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for batch_x, batch_ids, batch_y_risk, batch_y_residual in val_loader:
                batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
                risk_logits, residual_pred = model_lstm(batch_x, batch_ids)
                loss = best_params_lstm['risk_weight'] * nn.CrossEntropyLoss()(risk_logits, batch_y_risk) + nn.MSELoss()(residual_pred, batch_y_residual)
                epoch_val_loss += loss.item()
        val_losses.append(epoch_val_loss / len(val_loader))

        if (epoch + 1) % 10 == 0:
            logging.info(f"Epoch {epoch+1}/50, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

            
    torch.save(model_lstm.state_dict(), os.path.join(model_dir, f'lstm_model_{experiment_name}.pth'))

    # ===== Evaluasi dan Prediksi =====
    model_lstm.eval()
    with torch.no_grad():
        lstm_residual_pred = model_lstm(torch.tensor(X_test_aug).to(device), torch.tensor(X_test_ids).to(device))[1].cpu().numpy()
    
    arima_test_pred_aligned = arima_test_pred.values[timesteps:][:len(lstm_residual_pred)]
    hybrid_test_pred = arima_test_pred_aligned + lstm_residual_pred
    y_true_test = y_test[:, -1]
    
    hybrid_mae = mean_absolute_error(y_true_test, hybrid_test_pred)
    hybrid_rmse = np.sqrt(mean_squared_error(y_true_test, hybrid_test_pred))
    # MODIFIKASI: MAPE tidak dihitung
    # hybrid_mape = mean_absolute_percentage_error(y_true_test, hybrid_test_pred)
    logging.info(f"Kinerja Hybrid {experiment_name}:\n - MAE: {hybrid_mae:.4f}\n - RMSE: {hybrid_rmse:.4f}")

    # ===== Peramalan Masa Depan =====
    test_dates_for_df = test_data['Date'].iloc[timesteps:].iloc[:len(y_true_test)]
    test_student_ids_for_df = test_data['Student ID'].iloc[timesteps:].iloc[:len(y_true_test)]
    predictions_df = pd.DataFrame({'Date': test_dates_for_df, 'Student ID': test_student_ids_for_df, 'Predicted Stress Level': hybrid_test_pred, 'Actual Stress Level': y_true_test})
    
    full_data_with_residuals = pd.concat([train_data, test_data]).sort_values(by=['Student ID', 'Date'])
    
    future_forecasts = forecast_future(full_data_with_residuals, arima_stress_fit, model_lstm, le_student, feature_forecasters, timesteps, 10, feature_cols, primary_feature_cols)

    # ===== Plotting =====
    logging.info("Membuat grafik dan file CSV untuk visualisasi...")
    
    # MODIFIKASI: Membuat plot loss curve
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title(f'Training & Validation Loss Curve\nEksperimen: {experiment_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(model_dir, f'plot_loss_curve_{experiment_name}.png'))
    plt.close()
    
    # MODIFIKASI: Buat DataFrame untuk menyimpan semua data visualisasi
    visualization_data = []
    
    # 1. Tambahkan data aktual dari data_exp
    for student_id in data_exp['Student ID'].unique():
        student_actual_data = data_exp[data_exp['Student ID'] == student_id][['Date', 'Student ID', 'Stress Level (GSR)']]
        for _, row in student_actual_data.iterrows():
            visualization_data.append({
                'Date': row['Date'],
                'Student ID': row['Student ID'],
                'Actual Stress Level': row['Stress Level (GSR)'],
                'Predicted Stress Level': np.nan,
                'Residuals': np.nan,
                'Forecasted Stress Level': np.nan
            })
    
    # 2. Tambahkan data prediksi dari predictions_df
    for _, row in predictions_df.iterrows():
        visualization_data.append({
            'Date': row['Date'],
            'Student ID': row['Student ID'],
            'Actual Stress Level': row['Actual Stress Level'],
            'Predicted Stress Level': row['Predicted Stress Level'],
            'Residuals': row['Actual Stress Level'] - row['Predicted Stress Level'],
            'Forecasted Stress Level': np.nan
        })
    
    # 3. Tambahkan data peramalan dari future_forecasts (jika ada)
    if future_forecasts is not None:
        for _, row in future_forecasts.iterrows():
            visualization_data.append({
                'Date': row['Date'],
                'Student ID': row['Student ID'],
                'Actual Stress Level': np.nan,
                'Predicted Stress Level': np.nan,
                'Residuals': np.nan,
                'Forecasted Stress Level': row['Forecasted Stress Level']
            })
    
    # Buat DataFrame dari visualization_data
    visualization_df = pd.DataFrame(visualization_data)
    visualization_df = visualization_df.sort_values(by=['Student ID', 'Date']).reset_index(drop=True)
    
    # Simpan ke file CSV
    csv_path = os.path.join(model_dir, f'visualization_data_{experiment_name}.csv')
    visualization_df.to_csv(csv_path, index=False)
    logging.info(f"Data visualisasi disimpan ke {csv_path}")
    
    # Plotting per siswa (tetap dipertahankan)
    for student_id in data_exp['Student ID'].unique():
        plt.figure(figsize=(15, 7))
        student_actual_data = data_exp[data_exp['Student ID'] == student_id]
        plt.plot(student_actual_data['Date'], student_actual_data['Stress Level (GSR)'], label='Actual Stress Level', color='blue', linestyle='-')
        
        student_pred_data = predictions_df[predictions_df['Student ID'] == student_id]
        if not student_pred_data.empty:
            plt.plot(student_pred_data['Date'], student_pred_data['Predicted Stress Level'], label=f'Hybrid Prediction ({experiment_name})', color='green', linestyle='--')
            residuals = student_pred_data['Actual Stress Level'] - student_pred_data['Predicted Stress Level']
            plt.plot(student_pred_data['Date'], residuals, label='Residuals', color='red', linestyle=':')
        
        if future_forecasts is not None:
            student_forecast_data = future_forecasts[future_forecasts['Student ID'] == student_id]

            if not student_forecast_data.empty:
                if not student_pred_data.empty:
                    # last_pred_date = student_pred_data['Date'].last()
                    last_pred_date = student_pred_data['Date'].iloc[-1]

                    last_pred_value = student_pred_data['Predicted Stress Level'].values[-1]
                    combined_forecast_dates = pd.concat([pd.Series([last_pred_date]), pd.Series(student_forecast_data['Date'])]).reset_index(drop=True)
                    combined_forecast_values = pd.concat([pd.Series([last_pred_value]), pd.Series(student_forecast_data['Forecasted Stress Level'])]).reset_index(drop=True)
                    plt.plot(combined_forecast_dates, combined_forecast_values, color='green', linestyle='--')
                
                plt.plot(student_forecast_data['Date'], student_forecast_data['Forecasted Stress Level'], color='orange', marker='o', linestyle='--', label='Dynamic Forecast')
        
        plt.title(f'Stress Level Prediction for Student {student_id}\nEksperimen: {experiment_name}')
        plt.xlabel('Date')
        plt.ylabel('Stress Level (GSR)')
        plt.legend()
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=3))
        plt.gcf().autofmt_xdate(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(model_dir, f'plot_{experiment_name}_student_{student_id}.png'))
        plt.close()

    logging.info(f"===== EKSPERIMEN SELESAI: {experiment_name} =====")
    # MODIFIKASI: Menghapus mape_stress dari return dictionary
    return {'mae_stress': hybrid_mae, 'rmse_stress': hybrid_rmse, 'predictions': predictions_df, 'future_forecasts': future_forecasts}

# =============================================================================
# BAGIAN 4: EKSEKUSI EKSPERIMEN
# =============================================================================

if __name__ == "__main__":
    try:
        data_main = pd.read_csv('dataset_baru.csv')
        # Lakukan rekayasa fitur dasar sekali saja
        data_main = data_main.sort_values(by=['Student ID', 'Date']).reset_index(drop=True)
        data_main['lag_1_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].shift(1)
        data_main['lag_1_sleep'] = data_main.groupby('Student ID')['Sleep Hours'].shift(1)
        data_main['rolling_mean_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].rolling(window=7, min_periods=1).mean().reset_index(0,drop=True)
        data_main['rolling_std_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].rolling(window=7, min_periods=1).std().reset_index(0,drop=True)
        data_main.ffill(inplace=True)
        data_main.bfill(inplace=True)

    except FileNotFoundError:
        print("Error: File 'dataset_baru.csv' tidak ditemukan.")
        exit()

    # Definisikan set fitur
    features_full = ['Sleep Hours', 'Anxiety Level', 'Mood Score', 'Attendance Status (Data)', 'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']
    primary_features_full = ['Sleep Hours', 'Anxiety Level', 'Mood Score', 'Attendance Status (Data)']
    
    features_reduced = ['Anxiety Level', 'Mood Score']
    primary_features_reduced = ['Anxiety Level', 'Mood Score']

    # Konfigurasi untuk 4 eksperimen
    experiments = [
        {
            "name": "1_Hybrid_NoAttn_FullFeatures",
            "dir": "hasil_eksperimen baru_1_hybrid_no_attn_full_features",
            "features": features_full,
            "primary_features": primary_features_full,
            "use_attention": False
        },
        {
            "name": "2_Hybrid_WithAttn_FullFeatures",
            "dir": "hasil_eksperimen baru_2_hybrid_with_attn_full_features",
            "features": features_full,
            "primary_features": primary_features_full,
            "use_attention": True
        },
        {
            "name": "3_Hybrid_NoAttn_ReducedFeatures",
            "dir": "hasil_eksperimen baru_3_hybrid_no_attn_reduced_features",
            "features": features_reduced,
            "primary_features": primary_features_reduced,
            "use_attention": False
        },
        {
            "name": "4_Hybrid_WithAttn_ReducedFeatures",
            "dir": "hasil_eksperimen baru_4_hybrid_with_attn_reduced_features",
            "features": features_reduced,
            "primary_features": primary_features_reduced,
            "use_attention": True
        }
    ]

    total_start_time = time.time()
    
    for exp in experiments:
        print("\n" + "="*80)
        print(f"MEMULAI EKSPERIMEN: {exp['name']}")
        print("="*80)

        exp_start_time = time.time()
        
        metrics = run_experiment(
            data=data_main,
            experiment_name=exp['name'],
            model_dir=exp['dir'],
            feature_cols=exp['features'],
            primary_feature_cols=exp['primary_features'],
            use_attention=exp['use_attention']
        )
        
        exp_end_time = time.time()
        duration = exp_end_time - exp_start_time
        hours, rem = divmod(duration, 3600)
        minutes, seconds = divmod(rem, 60)

        print(f"\n--- HASIL UNTUK EKSPERIMEN: {exp['name']} ---")
        if metrics:
            print(f"Metrik Akhir (Hybrid pada Set Uji):")
            print(f" - MAE  (Tingkat Stres): {metrics['mae_stress']:.4f}")
            print(f" - RMSE (Tingkat Stres): {metrics['rmse_stress']:.4f}")
            # MODIFIKASI: Menghapus print MAPE
            # print(f" - MAPE (Tingkat Stres): {metrics['mape_stress']:.2f}%")
            if metrics['future_forecasts'] is not None:
                print("\nPeramalan Masa Depan Dinamis (Contoh):")
                print(metrics['future_forecasts'].head())
            print(f"\nGrafik prediksi, peramalan, dan kurva loss telah disimpan di folder '{exp['dir']}'") # MODIFIKASI: Menambahkan info kurva loss
        print(f"Waktu Eksekusi Eksperimen: {int(hours)} jam, {int(minutes)} menit, {seconds:.2f} detik")
        print("="*80)
        
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    total_hours, total_rem = divmod(total_duration, 3600)
    total_minutes, total_seconds = divmod(total_rem, 60)
    print(f"\nTotal Waktu Eksekusi Semua Eksperimen: {int(total_hours)} jam, {int(total_minutes)} menit, {total_seconds:.2f} detik")


MEMULAI EKSPERIMEN: 1_Hybrid_NoAttn_FullFeatures

 Proses tuning ARIMA dimulai...
 - ARIMA(0,0,0) → MAE: 0.2509
 - ARIMA(0,0,1) → MAE: 0.2509
 - ARIMA(0,0,2) → MAE: 0.2509
 - ARIMA(0,0,3) → MAE: 0.2509
 - ARIMA(0,0,4) → MAE: 0.2509
 - ARIMA(0,1,0) → MAE: 0.3456
 - ARIMA(0,1,1) → MAE: 0.2508
 - ARIMA(0,1,2) → MAE: 0.2508
 - ARIMA(0,1,3) → MAE: 0.2508
 - ARIMA(0,1,4) → MAE: 0.2508
 - ARIMA(0,2,0) → MAE: 312.1551
 - ARIMA(0,2,1) → MAE: 0.4441
 - ARIMA(0,2,2) → MAE: 0.2538
 - ARIMA(0,2,3) → MAE: 0.4975
 - ARIMA(0,2,4) → MAE: 0.6869
 - ARIMA(1,0,0) → MAE: 0.2509
 - ARIMA(1,0,1) → MAE: 0.2509
 - ARIMA(1,0,2) → MAE: 0.2509
 - ARIMA(1,0,3) → MAE: 0.2509
 - ARIMA(1,0,4) → MAE: 0.2509
 - ARIMA(1,1,0) → MAE: 0.3140
 - ARIMA(1,1,1) → MAE: 0.2508
 - ARIMA(1,1,2) → MAE: 0.2508
 - ARIMA(1,1,3) → MAE: 0.2508
 - ARIMA(1,1,4) → MAE: 0.2508
 - ARIMA(1,2,0) → MAE: 232.0128
 - ARIMA(1,2,1) → MAE: 0.3684
 - ARIMA(1,2,2) → MAE: 0.6106
 - ARIMA(1,2,3) → MAE: 0.2514
 - ARIMA(1,2,4) → MAE: 0.2513
 - ARIMA(2,0,

[I 2025-06-24 15:23:10,315] A new study created in memory with name: no-name-cdc67c21-7360-468c-a9de-fc7dfb4725d6
[I 2025-06-24 15:29:00,396] Trial 0 finished with value: 0.6063584685325623 and parameters: {'hidden_dim': 72, 'dropout_rate': 0.2803579230020786, 'lr': 0.0003282239857402389, 'risk_weight': 0.808761666534412, 'embed_dim': 11}. Best is trial 0 with value: 0.6063584685325623.
[I 2025-06-24 15:29:04,811] Trial 3 finished with value: 1.1481482982635498 and parameters: {'hidden_dim': 88, 'dropout_rate': 0.23926029694173978, 'lr': 0.0050006969910464565, 'risk_weight': 1.5390869036247872, 'embed_dim': 14}. Best is trial 0 with value: 0.6063584685325623.
[I 2025-06-24 15:29:07,503] Trial 6 finished with value: 0.8044189810752869 and parameters: {'hidden_dim': 80, 'dropout_rate': 0.3266743281315773, 'lr': 0.0004603207174744077, 'risk_weight': 1.043690635435045, 'embed_dim': 40}. Best is trial 0 with value: 0.6063584685325623.
[I 2025-06-24 15:30:06,279] Trial 2 finished with value:


📌 Hyperparameter LSTM terbaik dari tuning:
 - hidden_dim: 72
 - dropout_rate: 0.2803579230020786
 - lr: 0.0003282239857402389
 - risk_weight: 0.808761666534412
 - embed_dim: 11

--- HASIL UNTUK EKSPERIMEN: 1_Hybrid_NoAttn_FullFeatures ---
Metrik Akhir (Hybrid pada Set Uji):
 - MAE  (Tingkat Stres): 0.0233
 - RMSE (Tingkat Stres): 0.0285

Peramalan Masa Depan Dinamis (Contoh):
        Date  Student ID  Forecasted Stress Level
0 2024-12-31           1                 0.525178
1 2025-01-01           1                 0.896470
2 2025-01-02           1                 0.836251
3 2025-01-03           1                 0.714216
4 2025-01-04           1                 0.880792

Grafik prediksi, peramalan, dan kurva loss telah disimpan di folder 'hasil_eksperimen baru_1_hybrid_no_attn_full_features'
Waktu Eksekusi Eksperimen: 0 jam, 21 menit, 59.62 detik

MEMULAI EKSPERIMEN: 2_Hybrid_WithAttn_FullFeatures

 Proses tuning ARIMA dimulai...
 - ARIMA(0,0,0) → MAE: 0.2509
 - ARIMA(0,0,1) → MAE: 0.

[I 2025-06-24 15:45:22,011] A new study created in memory with name: no-name-722f4174-945d-4db7-a6d6-73c950f4db18
[I 2025-06-24 15:52:44,290] Trial 0 finished with value: 1.0687808990478516 and parameters: {'hidden_dim': 64, 'dropout_rate': 0.30952491261654075, 'lr': 0.0037983000541339236, 'risk_weight': 1.4283239445198985, 'embed_dim': 21, 'num_heads': 8}. Best is trial 0 with value: 1.0687808990478516.
[I 2025-06-24 15:53:15,902] Trial 5 finished with value: 0.7746705412864685 and parameters: {'hidden_dim': 96, 'dropout_rate': 0.4856737922430505, 'lr': 0.00041956474394577243, 'risk_weight': 1.0288423975524237, 'embed_dim': 28, 'num_heads': 4}. Best is trial 5 with value: 0.7746705412864685.
[I 2025-06-24 15:53:24,416] Trial 6 finished with value: 0.8445502519607544 and parameters: {'hidden_dim': 120, 'dropout_rate': 0.2470502904200109, 'lr': 0.003963181005962017, 'risk_weight': 1.1246617432035289, 'embed_dim': 26, 'num_heads': 2}. Best is trial 5 with value: 0.7746705412864685.
[I 20


📌 Hyperparameter LSTM terbaik dari tuning:
 - hidden_dim: 160
 - dropout_rate: 0.371578150372286
 - lr: 0.0049056784269915265
 - risk_weight: 0.8404431201749435
 - embed_dim: 34
 - num_heads: 4

--- HASIL UNTUK EKSPERIMEN: 2_Hybrid_WithAttn_FullFeatures ---
Metrik Akhir (Hybrid pada Set Uji):
 - MAE  (Tingkat Stres): 0.0367
 - RMSE (Tingkat Stres): 0.0466

Peramalan Masa Depan Dinamis (Contoh):
        Date  Student ID  Forecasted Stress Level
0 2024-12-31           1                 0.202460
1 2025-01-01           1                 0.852869
2 2025-01-02           1                 0.788973
3 2025-01-03           1                 0.201564
4 2025-01-04           1                 0.771974

Grafik prediksi, peramalan, dan kurva loss telah disimpan di folder 'hasil_eksperimen baru_2_hybrid_with_attn_full_features'
Waktu Eksekusi Eksperimen: 0 jam, 26 menit, 9.11 detik

MEMULAI EKSPERIMEN: 3_Hybrid_NoAttn_ReducedFeatures

 Proses tuning ARIMA dimulai...
 - ARIMA(0,0,0) → MAE: 0.2509
 - A

[I 2025-06-24 16:11:13,598] A new study created in memory with name: no-name-3ae1f53d-a87c-4278-b6a2-e37b2e98cad3
[I 2025-06-24 16:16:27,542] Trial 3 finished with value: 0.941009521484375 and parameters: {'hidden_dim': 88, 'dropout_rate': 0.3128087687342327, 'lr': 0.0028836143234617444, 'risk_weight': 1.270247181888909, 'embed_dim': 13}. Best is trial 3 with value: 0.941009521484375.
[I 2025-06-24 16:16:46,802] Trial 5 finished with value: 0.6351321339607239 and parameters: {'hidden_dim': 96, 'dropout_rate': 0.3144555093846444, 'lr': 0.001557900823174226, 'risk_weight': 0.8518990439562047, 'embed_dim': 16}. Best is trial 5 with value: 0.6351321339607239.
[I 2025-06-24 16:16:47,917] Trial 0 finished with value: 1.0826919078826904 and parameters: {'hidden_dim': 96, 'dropout_rate': 0.2698953992970763, 'lr': 0.00013390709521056605, 'risk_weight': 1.426840507250898, 'embed_dim': 27}. Best is trial 5 with value: 0.6351321339607239.
[I 2025-06-24 16:17:09,621] Trial 7 finished with value: 1.


📌 Hyperparameter LSTM terbaik dari tuning:
 - hidden_dim: 96
 - dropout_rate: 0.3144555093846444
 - lr: 0.001557900823174226
 - risk_weight: 0.8518990439562047
 - embed_dim: 16

--- HASIL UNTUK EKSPERIMEN: 3_Hybrid_NoAttn_ReducedFeatures ---
Metrik Akhir (Hybrid pada Set Uji):
 - MAE  (Tingkat Stres): 0.0234
 - RMSE (Tingkat Stres): 0.0303

Peramalan Masa Depan Dinamis (Contoh):
        Date  Student ID  Forecasted Stress Level
0 2024-12-31           1                 0.640553
1 2025-01-01           1                 0.866719
2 2025-01-02           1                 0.804636
3 2025-01-03           1                 0.731241
4 2025-01-04           1                 0.805133

Grafik prediksi, peramalan, dan kurva loss telah disimpan di folder 'hasil_eksperimen baru_3_hybrid_no_attn_reduced_features'
Waktu Eksekusi Eksperimen: 0 jam, 21 menit, 28.93 detik

MEMULAI EKSPERIMEN: 4_Hybrid_WithAttn_ReducedFeatures

 Proses tuning ARIMA dimulai...
 - ARIMA(0,0,0) → MAE: 0.2509
 - ARIMA(0,0,1) 

[I 2025-06-24 16:32:46,891] A new study created in memory with name: no-name-38c758d8-e48a-462e-b2f1-fb1fcad46182
[I 2025-06-24 16:40:07,547] Trial 7 finished with value: 1.2477144002914429 and parameters: {'hidden_dim': 64, 'dropout_rate': 0.23815693380341355, 'lr': 0.0008814687720308008, 'risk_weight': 1.671791547763339, 'embed_dim': 16, 'num_heads': 4}. Best is trial 7 with value: 1.2477144002914429.
[I 2025-06-24 16:40:54,516] Trial 2 finished with value: 1.3142988681793213 and parameters: {'hidden_dim': 96, 'dropout_rate': 0.4914605452607248, 'lr': 0.0011955875904742226, 'risk_weight': 1.7691294731653135, 'embed_dim': 43, 'num_heads': 8}. Best is trial 7 with value: 1.2477144002914429.
[I 2025-06-24 16:41:26,043] Trial 6 finished with value: 1.1068897247314453 and parameters: {'hidden_dim': 136, 'dropout_rate': 0.24459019486123115, 'lr': 0.0010613378600294132, 'risk_weight': 1.4876160481188698, 'embed_dim': 38, 'num_heads': 8}. Best is trial 6 with value: 1.1068897247314453.
[I 20


📌 Hyperparameter LSTM terbaik dari tuning:
 - hidden_dim: 144
 - dropout_rate: 0.3820001443286044
 - lr: 0.00040648639300275324
 - risk_weight: 0.9295214029538166
 - embed_dim: 48
 - num_heads: 2

--- HASIL UNTUK EKSPERIMEN: 4_Hybrid_WithAttn_ReducedFeatures ---
Metrik Akhir (Hybrid pada Set Uji):
 - MAE  (Tingkat Stres): 0.0313
 - RMSE (Tingkat Stres): 0.0395

Peramalan Masa Depan Dinamis (Contoh):
        Date  Student ID  Forecasted Stress Level
0 2024-12-31           1                 0.431749
1 2025-01-01           1                 0.791111
2 2025-01-02           1                 0.729630
3 2025-01-03           1                 0.523448
4 2025-01-04           1                 0.687164

Grafik prediksi, peramalan, dan kurva loss telah disimpan di folder 'hasil_eksperimen baru_4_hybrid_with_attn_reduced_features'
Waktu Eksekusi Eksperimen: 0 jam, 25 menit, 2.73 detik

Total Waktu Eksekusi Semua Eksperimen: 1 jam, 34 menit, 40.49 detik
